In [10]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from scipy import stats

In [5]:
#State dictionary

states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [26]:
#Cleaning the university towns file

def get_list_of_university_towns():
    wiki=pd.read_csv('university_towns.txt',sep='[\t]',header=None,engine='python').rename(columns={0:'State'})
    wiki['RegionName']=wiki.State.apply(lambda x:x)

    #Cleaning state names

    #If the state name does not have the "[edit]", it will be substituted by NaN
    wiki['State'].loc[~wiki['State'].str.contains('[edit]',regex=False)]=np.nan
    #Removing brakets, whatever comes after the brakets and extra-spaces
    wiki['State']=(wiki['State'].str.replace(r'\[.*','')).str.strip()
    #Substituing all the NaN by the state name
    wiki=wiki.ffill(axis=0)

    #Cleaning region names

    #If the name contains "[edit]", it will be substituted by NaN
    wiki['RegionName'].loc[wiki['RegionName'].str.contains('[edit]',regex=False)]=np.nan
    #Removing non-alphanumeric characters, whatever starts with a parenthesis and extra-spaces
    wiki['RegionName']=(wiki['RegionName'].str.replace(r'W*\(.*','')).str.strip()
    #Dropping the rows that will have NaN
    wiki=(wiki.dropna()).reset_index(drop=True)
    
    return wiki

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [40]:
#Getting the start of the recession (I am using the GDP in billions of dollars here)
def get_recession_start():
    gdp=pd.read_excel('gdplev.xls',skiprows=219,usecols=[4,5])
    gdp.columns=['Quarter','GDP']

    df=gdp[(gdp['GDP']>gdp['GDP'].shift(-1)) & (gdp['GDP'].shift(-1)>gdp['GDP'].shift(-2))].copy()
    
    return df.iloc[0,0]

get_recession_start()

'2008q3'

In [45]:
#Getting the end of the recession
def get_recession_end():
    gdp=pd.read_excel('gdplev.xls',skiprows=219,usecols=[4,5])
    gdp.columns=['Quarter','GDP']
    gdp['Year']=gdp['Quarter'].str[:4].astype(int)
    df=gdp[(gdp['GDP']>gdp['GDP'].shift(1)) & (gdp['GDP'].shift(1)>gdp['GDP'].shift(2))].copy()
    
    #Limiting df to be after 2008 (since we know that it started in 2008)
    df=df.loc[df['Year']>2008]
    
    return df.iloc[0,0]

get_recession_end()

'2009q4'

In [61]:
def get_recession_bottom():
    #Getting the bottom of the recession
    gdp=pd.read_excel('gdplev.xls',skiprows=219,usecols=[4,5])
    gdp.columns=['Quarter','GDP']

    #Limiting the data
    start=get_recession_start()
    end=get_recession_end()
    df=(gdp[(gdp['Quarter']>=start) & (gdp['Quarter']<=end)].copy())
    df.set_index('Quarter',inplace=True)
    
    return df['GDP'].idxmin()

get_recession_bottom()

'2009q2'

In [78]:
#Preparing the Zillow data

def convert_housing_data_to_quarters():
    zillow=pd.read_csv('City_Zhvi_AllHomes.csv')

    #Keeping only the necessary columns
    zillow.drop(zillow.columns[5:51],inplace=True,axis=1)

    #Creating the data where we will aggregate the monthly columns into quarterly columns
    mean_data=(zillow.drop(zillow.columns[1:5],axis=1).copy()).set_index('RegionID')
    mean_data=mean_data.groupby(pd.PeriodIndex(mean_data.columns,freq='q'),axis=1).mean()

    #Formatting column names
    mean_data.columns=mean_data.columns.strftime('%Yq%q')

    #Reseting the index to prepare for merging the data back
    mean_data=mean_data.reset_index()

    #Keeping only a few columns from zillow to merge into the aggregated data
    zillow.drop(zillow.columns[3:],inplace=True,axis=1)

    #Merging the two datasets

    final_data=pd.merge(mean_data,zillow,how='inner',on='RegionID')
    final_data=final_data.drop('RegionID',axis=1)

    #Cleaning-up the state names
    final_data['State']=final_data['State'].map(states)

    final_data.set_index(['State','RegionName'],inplace=True)
    
    return final_data

convert_housing_data_to_quarters()    

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [94]:
#Running the t-test

def run_ttest():
    convert_housing_data_to_quarters()  

    get_list_of_university_towns()
    wiki['Uni Flag']=1

    df=pd.merge(final_data,wiki,how='left',on=['State','RegionName'])
    df['Uni Flag']=df['Uni Flag'].fillna(0)

    start_r=get_recession_start()
    bottom=get_recession_bottom()

    #We want the quarter before the recession started rather than the quarter when it started
    b_start=df.columns[df.columns.get_loc(start_r)-1]

    #Finding out what was the change (ratio) from the start and bottom of the recession
    df['Change']=df[b_start]/df[bottom]

    #Separating the data into two groups
    uni=df.where(df['Uni Flag']==1.0).copy().dropna()['Change']
    non_uni=df.where(df['Uni Flag']==0.0).copy().dropna()['Change']

    #Getting the results
    pval=list(stats.ttest_ind(uni,non_uni))[1]
    different=True if pval<0.01 else False
    better="university town" if uni.mean()<non_uni.mean() else "non-university town"
    
    return (different,pval,better)

run_ttest()


(True, 0.005164866327919532, 'university town')